# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

df = pd.read_csv('previsao_de_renda.csv', parse_dates=['data_ref'])
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [2]:
df = pd.read_csv('previsao_de_renda.csv')

# Exibir as colunas presentes no DataFrame
print(df.columns)


Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')


In [4]:
y, x = patsy.dmatrices('''

                       np.log(renda) ~

                               C(sexo, Treatment(0))

                               + C(tipo_renda, Treatment(0))

                               + C(educacao, Treatment(2))

                               + C(estado_civil, Treatment(0))

                               + C(tipo_residencia, Treatment(1))

                               + C(posse_de_veiculo, Treatment(0))

                               + C(posse_de_imovel, Treatment(1))

                               + qtd_filhos

                               + idade

                               + tempo_emprego

                               + qt_pessoas_residencia

                       ''', data = df)
sm.OLS(y, x).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Mon, 07 Aug 2023   Prob (F-statistic):               0.00
Time:                        21:07:36   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo, Treatment(0))[T.M]                            0.7874      0.015     53.723      0.000       0.759       0.816
C(tipo_renda, Treatment(0))[T.Bolsista]               0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment(0))[T.Empresário]             0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment(0))[T.Pensionista]           -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment(0))[T.Servidor público]       0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]            0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]        0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment(0))[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment(0))[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment(0))[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
C(posse_de_veiculo, Treatment(0))[T.True]             0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]            -0.0829      0.014     -5.926      0.000      -0.110      -0.055
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy

# Carregar os dados e pré-processá-los
df = pd.read_csv('previsao_de_renda.csv', parse_dates=['data_ref'])
df.drop('Unnamed: 0', axis=1, inplace=True)

# Definir a fórmula do modelo
formula = '''
    np.log(renda) ~
        C(sexo, Treatment(0)) +
        C(tipo_renda, Treatment(0)) +
        C(educacao, Treatment(2)) +
        C(estado_civil, Treatment(0)) +
        C(tipo_residencia, Treatment(1)) +
        C(posse_de_veiculo, Treatment(0)) +
        C(posse_de_imovel, Treatment(1)) +
        qtd_filhos +
        idade +
        tempo_emprego +
        qt_pessoas_residencia
'''

# Ajustar o modelo inicial
y, X = patsy.dmatrices(formula, data=df, return_type='dataframe')
model = sm.OLS(y, X).fit()

# Função para remoção das variáveis menos significativas
def backward_elimination(model, X):
    p_values = pd.Series(model.pvalues, index=X.columns)
    max_p_value = p_values.idxmax()
    if p_values[max_p_value] > 0.05:
        print(f"Removing '{max_p_value}' (p-value: {p_values[max_p_value]:.4f})")
        X = X.drop(max_p_value, axis=1)
        model = sm.OLS(y, X).fit()
        return backward_elimination(model, X)
    else:
        return model, X

# Executar a eliminação backward
final_model, final_X = backward_elimination(model, X)

# Exibir o resumo do modelo final
print(final_model.summary())


Removing 'C(educacao, Treatment(2))[T.Primário]' (p-value: 0.8444)
Removing 'C(tipo_residencia, Treatment(1))[T.Governamental]' (p-value: 0.7442)
Removing 'C(tipo_residencia, Treatment(1))[T.Com os pais]' (p-value: 0.5175)
Removing 'C(tipo_residencia, Treatment(1))[T.Aluguel]' (p-value: 0.4269)
Removing 'C(tipo_residencia, Treatment(1))[T.Comunitário]' (p-value: 0.4043)
Removing 'C(educacao, Treatment(2))[T.Pós graduação]' (p-value: 0.3971)
Removing 'C(educacao, Treatment(2))[T.Superior incompleto]' (p-value: 0.3663)
Removing 'C(tipo_renda, Treatment(0))[T.Bolsista]' (p-value: 0.3596)
Removing 'C(tipo_renda, Treatment(0))[T.Pensionista]' (p-value: 0.1983)
Removing 'C(tipo_residencia, Treatment(1))[T.Estúdio]' (p-value: 0.1889)
Removing 'C(estado_civil, Treatment(0))[T.União]' (p-value: 0.1849)
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS